<a href="https://colab.research.google.com/github/zelalemamera-stonybrook/projects-sandbox/blob/main/Multi_Layer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn


In [10]:
v = torch.as_tensor([[1,2,3], [1,2,3]])
v

tensor([[1, 2, 3],
        [1, 2, 3]])

In [26]:

class MLP(nn.Module):
  '''
  '''
  def __init__(self, input_dimension, density1, density2, density3, output_dimension, activate):
    super().__init__()
    self.layer1 = nn.Linear(input_dimension, density1)
    self.layer2 = nn.Linear(density1, density2)
    self.layer3 = nn.Linear(density2, density3)
    self.layer4 = nn.Linear(density3, output_dimension)
    self.activate = nn.ReLU()

  def forward(self, input_batch):
     output_batch = nn.SoftMax(self.layer4(self.activate(self.layer3(self.activate(self.layer2(self.activate(self.layer1(input_batch))))))))
     return output_batch


In [20]:
def train(model, training_data, target_data, epoch, loss_function, optimizer):
  '''
  '''
  model.train()
  loss_function = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters())
  for n in range(epoch):
    optimizer.zero_grad()
    loss_function(model.forward(training_data), target_data).backward()
    optimizer.step()
    print_model_accuracy(model, training_data, target_data)


In [25]:
def print_model_accuracy(model, training_data, target_data):
  '''
  '''


torch.Size([20, 10])
torch.Size([20])
torch.Size([10, 20])
torch.Size([10])
torch.Size([5, 10])
torch.Size([5])
torch.Size([3, 5])
torch.Size([3])
